###Biblioteca(s)...

In [ ]:
pip install geopy

In [ ]:
pip install xlrd

In [ ]:
# Importar bibliotecas...
import re
import os
import zipfile
import pandas as pd
from pyspark.sql.functions import *
from geopy.distance import great_circle

###Conectar...

In [ ]:
# Conectar-se ao Data Lake...
dbutils.fs.mount( 
    source = "wasbs://dados@engdadosmvp.blob.core.windows.net", 
    mount_point = "/mnt/dados",
    extra_configs = {"fs.azure.account.key.engdadosmvp.blob.core.windows.net":"7uTSqaaFzINeCJ27Kmk+LOlzlJANBf2uupuOoy8MdgC8yg2Gggkaq7F6aXqvcodWcOIwKNowmcsE+AStKlIeSA=="})

###Ler...

In [ ]:
display(dbutils.fs.ls("/mnt/dados/"))

#####github...

In [ ]:
df_coordenadas_dos_municipios = spark.read.format("csv")\
                                          .option("sep", ",")\
                                          .option("header", "true")\
                                          .option("inferSchema", "true")\
                                          .load("/mnt/dados/municipios.csv")

#####ibge...

In [ ]:
df_pop_censo_2022 = pd.read_excel('/dbfs/mnt/dados/POP2022_Municipios_20230622.xls',
                    skiprows=1,
                            dtype={
                                'COD. UF':str,
                                'COD. MUNIC':str,
                                'POPULAÇÃO':str
                            }                   
).dropna(thresh=2)

df_area_munic = pd.read_excel('/dbfs/mnt/dados/AR_BR_RG_UF_RGINT_MES_MIC_MUN_2022.xls', 
                      sheet_name='AR_BR_MUN_2022', 
                      usecols=[4, 6]
)

df_inflacao_acumulada = pd.read_csv("/dbfs/mnt/dados/20230827084323.csv",
                        sep = ';',
                        skiprows=2,
                        names=['ano', 'inflacao_acumulada']
).dropna()

df_taxa_desemprego_media = pd.read_csv("/dbfs/mnt/dados/20230827084621.csv",
                        sep = ';',
                        skiprows=1,
).dropna().transpose()

#####inmet...

In [ ]:
PASTA_CAMINHO = '/dbfs/mnt/dados/'
ARQUIVOS = ['2019', '2020', '2021', '2022']

df_estacoes_meteorologicas = pd.read_csv(f'{PASTA_CAMINHO}{"estacoes_meteorologicas_br.csv"}',
                             sep=';'
)

def processar_arquivos(caminho_arquivo):
    tabelas = []
    nomes_arquivos = []
    total_linhas = []

    with zipfile.ZipFile(caminho_arquivo) as z:
        for nome_arquivo_interno in z.namelist():
            if nome_arquivo_interno.endswith('.CSV'):
                with z.open(nome_arquivo_interno) as f:
                    df = pd.read_csv(
                        f,
                        sep=';',
                        skiprows=8,
                        encoding='cp1252',
                        usecols=[0, 2, 10],
                        dtype={0: 'str', 2: 'float', 10: 'float'},
                        decimal=','
                    )

                    df['Nome_Arquivo'] = nome_arquivo_interno
                    nomes_arquivos.append(nome_arquivo_interno)
                    tabelas.append(df)
                    total_linhas.append(len(df))

    return tabelas, nomes_arquivos, total_linhas 

def main():
    tabelas_total = []
    nomes_arquivos_total = []
    df_log_data = []

    for nome_arquivo in os.listdir(PASTA_CAMINHO):
        if any(ano in nome_arquivo for ano in ARQUIVOS) and nome_arquivo.endswith('.zip'):
            caminho_zip = os.path.join(PASTA_CAMINHO, nome_arquivo)
            tabelas, nomes_arquivos, total_linhas = processar_arquivos(caminho_zip)

            df_log_data.extend([{'nomes_arquivos_lidos': nome_arquivo, 'total_linhas': total_linhas_tabela}
                                for nome_arquivo, total_linhas_tabela in zip(nomes_arquivos, total_linhas)])

            tabelas_total.extend(tabelas)
            nomes_arquivos_total.extend(nomes_arquivos)

    df_log = pd.DataFrame(df_log_data)
    return tabelas_total

if __name__ == "__main__":
    tabelas_total = main()
    df_inmet = pd.concat(tabelas_total, ignore_index=True)

#####ufmg...

In [ ]:
df_moradores_rua = spark.read.format("csv")\
                                  .option("sep", ";")\
                                  .option("header", "true")\
                                  .option("inferSchema", "true")\
                                  .load("/mnt/dados/municipios_serie_historica_pop_rua.csv")

#####suas...

In [ ]:
df_pop_censo_2010 = spark.read.format("csv")\
                                          .option("sep", ";")\
                                          .option("header", "true")\
                                          .option("inferSchema", "true") \
                                          .load("/mnt/dados/Lista_Municípios_com_IBGE_Brasil_Versao_CSV.csv")

###ETL...

#####github...

In [ ]:

df_coord_municipios = df_coordenadas_dos_municipios.toPandas()

manter_colunas = ['ESTACAO','LATITUDE', 'UF', 'LONGITUDE', 
                     'dias_temp_13_ano', 'dias_precip_50mm_ano']  

df_unificado = pd.merge(df_estacoes_meteorologicas, resumo_temperatura_baixa, left_on='ESTACAO', right_on='Nome_Arquivo', how='left')
df_unificado = pd.merge(df_unificado, resumo_pluviosidade, left_on='ESTACAO', right_on='Nome_Arquivo', how='left')

df_unificado = df_unificado[manter_colunas]
df_unificado['ESTACAO'] = df_unificado['ESTACAO'].apply(lambda x: "ESTACAO METEOROLOGICA " + x)
df_unificado['LATITUDE'] = df_unificado['LATITUDE'].str.replace(',', '.')
df_unificado['LONGITUDE'] = df_unificado['LONGITUDE'].str.replace(',', '.')

def encontrar_estacao_mais_proxima(linha):
    distancias = df_unificado.apply(
        lambda r: great_circle((linha['latitude'], linha['longitude']), (r['LATITUDE'], r['LONGITUDE'])).kilometers,
        axis=1
    )
    indice_mais_proximo = distancias.idxmin()
    linha_mais_proxima = df_unificado.loc[indice_mais_proximo]
    return (
        linha_mais_proxima['ESTACAO'],
        linha_mais_proxima['dias_temp_13_ano'],
        linha_mais_proxima['dias_precip_50mm_ano']
    )

resultados = df_coord_municipios.apply(encontrar_estacao_mais_proxima, axis=1)
df_coord_municipios[[
    'ESTACAO', 'dias_temp_13_ano', 'dias_precip_50mm_ano'
]] = pd.DataFrame(
    resultados.to_list(),
    columns=['ESTACAO', 'dias_temp_13_ano', 'dias_precip_50mm_ano']
)

df_coord_municipios = spark.createDataFrame(df_coord_municipios)
df_coord_municipios = df_coord_municipios.withColumnRenamed('ESTACAO', 'estacoes_meteorologicas')
df_coord_municipios.createOrReplaceTempView('DIM_COORD_MUNICIPIOS')

#####ibge...

In [ ]:
df_pop_censo_2022['POPULAÇÃO'] = df_pop_censo_2022['POPULAÇÃO'].str.replace('.', '')
df_pop_censo_2022['POPULAÇÃO'] = df_pop_censo_2022['POPULAÇÃO'].str.split('(', expand=True)[0]
df_pop_censo_2022['POPULAÇÃO'] = df_pop_censo_2022['POPULAÇÃO'].astype(int)

df_pop_censo_2022["id_municipio"] = df_pop_censo_2022["COD. UF"] + df_pop_censo_2022["COD. MUNIC"]
df_pop_censo_2022.rename(columns={'POPULAÇÃO': 'populacao_censo_2022'}, inplace=True)
df_pop_censo_2022['populacao_censo_2022'] = df_pop_censo_2022['populacao_censo_2022'].astype(float).astype(int)
df_pop_censo_2022 = spark.createDataFrame(df_pop_censo_2022)
df_pop_censo_2022.createOrReplaceTempView('DIM_POP_CENSO_2022')

df_area_munic = spark.createDataFrame(df_area_munic)
df_area_munic.createOrReplaceTempView('DIM_AREA_MUNIC')

df_inflacao_acumulada['ano'] = df_inflacao_acumulada['ano'].str.split().str[-1]
df_inflacao_acumulada = spark.createDataFrame(df_inflacao_acumulada)
df_inflacao_acumulada.createOrReplaceTempView('DIM_INFLACAO_ACUMULADA')

df_taxa_desemprego_media = df_taxa_desemprego_media.reset_index()
df_taxa_desemprego_media.columns = ['ano', 'taxa_desemprego']
df_taxa_desemprego_media['ano'] = df_taxa_desemprego_media['ano'].str.split().str[-1]
df_taxa_desemprego_media = df_taxa_desemprego_media.groupby(['ano'])['taxa_desemprego'].mean().reset_index()
df_taxa_desemprego_media.rename(columns={'taxa_desemprego': 'taxa_desemprego_media'}, inplace=True)
df_taxa_desemprego_media = spark.createDataFrame(df_taxa_desemprego_media)
df_taxa_desemprego_media.createOrReplaceTempView('DIM_TAXA_DESEMPREGO_MEDIA')

#####inmet...

In [ ]:
def transformar_nome_arquivo(nome):
    return nome.split('_')[4]

def main():
    df_inmet['Data'] = pd.to_datetime(df_inmet['Data'])
    df_inmet['Ano'] = df_inmet['Data'].dt.year

    pluviosidade_por_dia_filtrado = (
        df_inmet.groupby(['Data', 'Ano', 'Nome_Arquivo'])['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)']
        .sum()
        .reset_index()
    )
    
    pluviosidade_por_dia_filtrado = pluviosidade_por_dia_filtrado[pluviosidade_por_dia_filtrado['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'] > 50]
    pluviosidade_por_dia_filtrado['Nome_Arquivo'] = pluviosidade_por_dia_filtrado['Nome_Arquivo'].apply(transformar_nome_arquivo)
    pluviosidade_por_dia_filtrado['id_dt_municipio'] = pluviosidade_por_dia_filtrado['Data'].astype(str) + pluviosidade_por_dia_filtrado['Nome_Arquivo']
    pluviosidade_por_dia_filtrado.drop_duplicates(subset='id_dt_municipio', inplace=True)
    
    resumo_pluviosidade = (
        pluviosidade_por_dia_filtrado.groupby(['Nome_Arquivo'])
        .size()
        .reset_index(name='dias_precip_>50mm')
    )
    
    resumo_pluviosidade['dias_precip_50mm_ano'] = (resumo_pluviosidade['dias_precip_>50mm']/len(ARQUIVOS)).astype(str)
    resumo_pluviosidade['dias_precip_50mm_ano'] = resumo_pluviosidade['dias_precip_50mm_ano'].str.replace('.', ',')

    temperatura_abaixo_de_13 = df_inmet[df_inmet['TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)'] < 13]
    temperatura_abaixo_de_13['Nome_Arquivo'] = temperatura_abaixo_de_13['Nome_Arquivo'].apply(transformar_nome_arquivo)
    temperatura_abaixo_de_13['Data'] = temperatura_abaixo_de_13['Data'].astype(str)
    temperatura_abaixo_de_13['id_dt'] = temperatura_abaixo_de_13['Data'] + temperatura_abaixo_de_13['Nome_Arquivo']
    temperatura_abaixo_de_13 = temperatura_abaixo_de_13.drop_duplicates(subset=['id_dt'])
    
    resumo_temperatura_baixa = (
          temperatura_abaixo_de_13.groupby(['Nome_Arquivo'])
          .size()
          .reset_index(name='dias_temp_<13')
    )

    resumo_temperatura_baixa['dias_temp_13_ano'] = (resumo_temperatura_baixa['dias_temp_<13'] / len(ARQUIVOS))
    limite_sup = resumo_temperatura_baixa['dias_temp_13_ano'].mean() + 3.5*resumo_temperatura_baixa['dias_temp_13_ano'].std()
    limite_inf = resumo_temperatura_baixa['dias_temp_13_ano'].mean() - 3.5*resumo_temperatura_baixa['dias_temp_13_ano'].std()
    resumo_temperatura_baixa = resumo_temperatura_baixa.loc[(resumo_temperatura_baixa['dias_temp_13_ano'] <= limite_sup) & (resumo_temperatura_baixa['dias_temp_13_ano'] >= limite_inf)]
    resumo_temperatura_baixa['dias_temp_13_ano'] = resumo_temperatura_baixa['dias_temp_13_ano'].astype(str)
    resumo_temperatura_baixa['dias_temp_13_ano'] = resumo_temperatura_baixa['dias_temp_13_ano'].str.replace('.', ',')

    return resumo_temperatura_baixa, resumo_pluviosidade

if __name__ == "__main__":
    resumo_temperatura_baixa, resumo_pluviosidade = main()
    main()

#####ufmg...

In [ ]:
colunas_para_excluir = ['_c0']
df_moradores_rua = df_moradores_rua.drop(*colunas_para_excluir)

df_moradores_rua = df_moradores_rua.withColumnRenamed('codigo_ibg', 'id_municipio')\
                                             .withColumnRenamed('freq', 'moradores_rua')\
                                             .withColumnRenamed('ano', 'ano_levantamento')

df_moradores_rua.createOrReplaceTempView("DIM_MORADORES_RUA")

#####suas...

In [ ]:
colunas_para_excluir = ['ConcatUF+Mun', 'IBGE', 'Capital', '_c9']
df_pop_censo_2010 = df_pop_censo_2010.drop(*colunas_para_excluir)

df_pop_censo_2010 = df_pop_censo_2010.withColumnRenamed('IBGE7', 'id_municipio')\
                             .withColumnRenamed('UF', 'uf')\
                             .withColumnRenamed('Município', 'municipio')\
                             .withColumnRenamed('Região', 'regiao')\
                             .withColumnRenamed('População 2010', 'populacao_censo_2010')\
                             .withColumnRenamed('Porte', 'porte')
                             
df_pop_censo_2010.createOrReplaceTempView("MUNICIPIOS")

###Modelo estrela...

In [ ]:
# Criação das tabelas dimensão...
df_pop_censo_2010_id = spark.sql(""" SELECT municipio
                          FROM MUNICIPIOS
                          ORDER BY municipio
""")

df_pop_censo_2010_id = df_pop_censo_2010_id.withColumn('id_municipio', monotonically_increasing_id()+1)
df_pop_censo_2010_id.createOrReplaceTempView('DIM_MUNICIPIOS')

df_regiao = spark.sql(""" SELECT DISTINCT regiao
                          FROM MUNICIPIOS
                          ORDER BY regiao
""")

df_regiao = df_regiao.withColumn('id_regiao', monotonically_increasing_id()+1)
df_regiao.createOrReplaceTempView('DIM_REGIAO')

df_estado = spark.sql(""" SELECT DISTINCT uf
                          FROM MUNICIPIOS
                          ORDER BY uf
""")

df_estado = df_estado.withColumn('id_uf', monotonically_increasing_id()+1)
df_estado.createOrReplaceTempView('DIM_UNIDADE_FEDERATIVA')

df_porte = spark.sql(""" SELECT DISTINCT porte
                          FROM MUNICIPIOS
                          ORDER BY porte
""")

df_porte = df_porte.withColumn('id_porte', monotonically_increasing_id()+1)
df_porte.createOrReplaceTempView('DIM_PORTE_MUNICIPIO')

In [ ]:
# Criação da tabela fato...
df_fato = spark.sql("""
                    SELECT 
                        M.id_municipio, U.id_uf, R.id_regiao, P.id_porte, C.latitude, C.longitude, AR.AR_MUN_2022,
                        C.dias_temp_13_ano, C.dias_precip_50mm_ano, C.estacoes_meteorologicas,
                        M.populacao_censo_2010, CXXII.populacao_censo_2022, MR.moradores_rua, INF.inflacao_acumulada, TD.taxa_desemprego_media, MR.ano_levantamento

                        FROM MUNICIPIOS M
                        LEFT JOIN DIM_MORADORES_RUA MR ON MR.id_municipio = M.id_municipio
                        LEFT JOIN DIM_COORD_MUNICIPIOS C ON C.codigo_ibge = M.id_municipio
                        LEFT JOIN DIM_POP_CENSO_2022 CXXII ON CXXII.id_municipio = M.id_municipio
                        LEFT JOIN DIM_AREA_MUNIC AR ON AR.CD_MUN = M.id_municipio
                        LEFT JOIN DIM_REGIAO R ON R.regiao = M.regiao
                        LEFT JOIN DIM_UNIDADE_FEDERATIVA U ON U.uf = M.uf
                        LEFT JOIN DIM_PORTE_MUNICIPIO P ON P.porte = M.porte
                        LEFT JOIN DIM_INFLACAO_ACUMULADA INF ON INF.ano = MR.ano_levantamento
                        LEFT JOIN DIM_TAXA_DESEMPREGO_MEDIA TD ON TD.ano = MR.ano_levantamento

                        ORDER BY M.id_municipio
""")

###Carga...

In [ ]:
def carga_sql(df, tabela):
    df.write.format("jdbc")\
            .mode("append")\
            .option("url", "jdbc:sqlserver://mouraxy.database.windows.net")\
            .option("port", "1433")\
            .option("user", "mouraxy")\
            .option("password", "**********")\
            .option("database", "bd_mouraxy")\
            .option("dbtable", tabela)\
            .save()

In [ ]:
# Enviar dados para o SQL...
#carga_sql(df_pop_censo_2010_id, "DW.DIM_MUNICIPIOS")
#carga_sql(df_regiao, "DW.DIM_REGIAO")
#carga_sql(df_estado, "DW.DIM_UNIDADE_FEDERATIVA")
#carga_sql(df_porte, "DW.DIM_PORTE_MUNICIPIO")
#carga_sql(df_fato, "DW.MORADORES_SITUACAO_RUA")

In [ ]:
# Enviar dados para o Data Lake...
df_fato.coalesce(1)\
    .write\
    .format('csv')\
    .mode('overwrite')\
    .option("header", "true")\
    .save('/mnt/dados/spark/')

def mover_arquivos(extensao, origem, destino, novo_nome):
    files = dbutils.fs.ls(origem)
    for i in range (0, len(files)):
        file = files[i].name
        if extensao in file:
            dbutils.fs.cp(files[i].path, destino+novo_nome+extensao)
            print('Origem: '+file+'\nDestino: '+destino+novo_nome+extensao)

mover_arquivos('.csv', '/mnt/dados/spark/', '/mnt/dados/dw/', 'FACT_POP_RUA')

Origem: part-00000-tid-2616322005217052032-98781045-9946-4705-83c3-0d2edbe09a85-79-1-c000.csv
Destino: /mnt/dados/dw/FACT_POP_RUA.csv
